# Matching FooDB to ASA24 Ingredient Descriptions
## Step 2: Clean FooDB's 'Content.csv'

__Required Input Files__

  - **Content.csv.bz2** - version 1, downloaded direct from foodb.ca
  - **Food.csv** - version 1, downloaded direct from foodb.ca  
 

__Information__  
This script reformats and fills in missing food descriptions in FoodB's Content.csv. Specifically, this code:
    
   1) Removes items derived from Pathbank (Lipid from Arabidopsis) and HMDB  
   2) Identifies missing common food descriptions  
   3) Add food descriptions from Food.csv
       - This can lose additional information about preparation. However, this is the best descriptor possible given the information provided.
   4) Identify which food descriptions are still missing and find closest description match.
       - Output: Food_updated.csv
   5) Update Content file to include food descriptions
   6) Export updated Content file
       - Output: Content_updated.csv
       
__Outputs__

  - **Content_updated.csv.bz2** - Food descriptions were added to all rows missing food descriptions in the original
  - **Food_updated.csv** - Food 554 was added to the original Food.csv

In [1]:
#Load modules
import os
import pandas as pd
import numpy as np
import re
import bz2

In [2]:
#Ensure working directory is the project folder
mapping = os.getcwd()
mapping

'/Users/stephanie.wilson/Desktop/SYNC/Scripts/FooDB_FNDDS'

In [3]:
#Load data
Content = pd.read_csv('FooDB/Content.csv.bz2', compression='bz2', low_memory=False)
Food = pd.read_csv('FooDB/Food.csv')

### 1) Remove rows from HMDB and Pathpank

We are looking for compounds specifically within foods. So we will remove: 
  - endogenously produced compounds from humans (HMDB)
  - lipids from arabidopsis (Pathbank)
  
Rows meeting these criteria were all predicted and did NOT contain any content values. 

In [78]:
#Remove data from Pathbank and HMDB
Content = Content[Content["orig_food_common_name"]!= 'Endogenous compounds from human (HMDB)']
Content = Content[Content['orig_food_common_name']!='Lipid from Arabidopsis (PathBank)']
Content.shape
#cuts numbers of rows down drastically

(1871970, 26)

### 2) Identify which rows do not have food descriptions

In [79]:
#Pull out rows with missing common food descriptions
missing = Content[Content['orig_food_common_name'].isnull()]
print(missing.shape[0], ' rows are missing common food descriptions')

971656  rows are missing common food descriptions


### 3) Add food descriptions from Food.csv

In [80]:
# Rename in Food.csv to match column in Content
Food = Food.rename(columns = {'id':'food_id'})

# Filter in specific rows
Food = Food.filter(['food_id', 'name'], axis = 1)

In [81]:
# Merge missing data with food.csv
missing = pd.merge(missing, Food, on = 'food_id', how = 'left')
print(missing.shape[0], ' rows are missing common food descriptions')

971656  rows are missing common food descriptions


In [82]:
# Replace orig_food_common_name (all missing) with matched name from Food.csv
missing['orig_food_common_name'] = missing['name']

In [83]:
# Drop the name column as we've transferred the names over to another column
missing.drop(columns=['name'], inplace=True)

### 4) Identify which food descriptions are still missing and rectify.

In [84]:
# What didn't get a name?
still_missing = missing[missing['orig_food_common_name'].isnull()]
print(still_missing.shape[0], 'rows are still missing common food descriptions')

1973 rows are still missing common food descriptions


In [85]:
# What food codes did not have a food description?
still_missing['food_id'].value_counts()
# food_id codes 554 and 16420 did not get a common name code

554      1970
16420       3
Name: food_id, dtype: int64

#### 4a) Resolve food_id 554

In [86]:
# What information do we have in FooDB Content on #554?
id554 = Content[(Content['food_id']==554)]
id554['orig_food_common_name'].value_counts()
# We can see that food_id 554 is equivalent to 'Owl, horned, flesh, raw (Alaska Native)'

Owl, horned, flesh, raw (Alaska Native)    11
Name: orig_food_common_name, dtype: int64

In [87]:
Food.loc[(Food['food_id']==554)]
# Food 554 does not exist in Food.csv. Add this entry

,food_id,name


In [88]:
# What information do we have in FooDB Content on #16420?
id16420 = Content[(Content['food_id']==16420)]
id16420
# We can see that food_id 554 is equivalent to 'Soy protein concentrate, aqueous washed'

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
1621939,1682964,2608,Compound,16420,NaN,"Soy protein concentrate, aqueous washed",NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,38.25,NaN,1
1621940,1682965,11831,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,52.81,NaN,1
1621941,1682966,698,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,4.94,NaN,1
1621942,1682967,5583,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,94.65,NaN,1


In [89]:
# Create Entry for 554, matches Food.csv col names
id554_entry = {'id':[554], 'name':['Owl, horned, flesh, raw (Alaska Native)']} 
id554_entry = pd.DataFrame(id554_entry)

# Reload Food.csv to make edits
Food_orig = pd.read_csv('FooDB/Food.csv')

# Add 554 to Foodb_food aka Food.csv
Food_orig = pd.concat([Food_orig, id554_entry], ignore_index = True)
Food_orig.to_csv('FooDB/Food_updated.csv', index = None, header = True)

#### 4b) Resolve food_id 16420

In [90]:
# Check if the code is a remnant from the database the compound was derived from and just didn't get converted.
id16420_orig = Content[(Content['orig_food_id']=='16420')]
id16420_orig
# Confirmed Code 16420 corresponds to FooDB's food codes 272 and 85

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
2945,2946,2,Nutrient,272,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:42 UTC,2019-12-11 21:08:52 UTC,NaN,NaN,58130.00,other,0
12462,13270,3,Nutrient,272,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,205,"Carbohydrate, by difference",...,DATABASE,NaN,NaN,2014-11-05 13:44:36 UTC,2019-12-11 21:08:52 UTC,NaN,NaN,30910.00,other,0
21441,23056,4,Nutrient,272,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,204,Total lipid (fat),...,DATABASE,NaN,NaN,2014-11-05 13:46:24 UTC,2019-12-11 21:08:52 UTC,NaN,NaN,460.00,other,0
29854,32237,5,Nutrient,272,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,291,"Fiber, total dietary",...,DATABASE,NaN,NaN,2014-11-05 13:48:02 UTC,2019-12-11 21:08:52 UTC,NaN,NaN,5500.00,other,0
41612,45207,11,Nutrient,272,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,626,16:1 undifferentiated,...,DATABASE,NaN,NaN,2014-11-05 13:50:20 UTC,2019-12-11 21:08:52 UTC,NaN,NaN,1.00,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819223,2268154,11682,Compound,85,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,613,16:0,...,DATABASE,NaN,NaN,2020-03-12 21:02:33 UTC,2020-03-12 21:02:33 UTC,NaN,NaN,38.00,other,1
1819224,2268155,2942,Compound,85,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,614,18:0,...,DATABASE,NaN,NaN,2020-03-12 21:02:33 UTC,2020-03-12 21:02:33 UTC,NaN,NaN,13.00,other,1
1819225,2268156,21595,Compound,85,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,645,"Fatty acids, total monounsaturated",...,DATABASE,NaN,NaN,2020-03-12 21:02:33 UTC,2020-03-12 21:02:33 UTC,NaN,NaN,79.00,other,1
1819226,2268157,21595,Compound,85,16420,"Soy protein concentrate, produced by acid wash",NaN,NaN,646,"Fatty acids, total polyunsaturated",...,DATABASE,NaN,NaN,2020-03-12 21:02:33 UTC,2020-03-12 21:02:33 UTC,NaN,NaN,201.00,other,1


In [91]:
# What is the usage like for unique food codes 272 and 85?
id16420_orig['food_id'].value_counts()

272    72
85     72
Name: food_id, dtype: int64

In [92]:
# What are the food descriptors for 272 and 85
Food.loc[(Food['food_id']==272)|(Food['food_id']==85)]
# Will choose to classify id 16420 with food_id 272 'Other soy product' moving forward as it is more similar to FNDDS usage

,food_id,name
84,85,Soy bean
270,272,Other soy product


In [93]:
still_missing
# Next steps:
# Replace 554 name with 'Owl, horned, flesh, raw (Alaska Native)'
# Replace 16420 id with 272 and name with 'Soy protein concentrate, aqueous washed'

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
13762,760493,30233,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:11 UTC,2015-05-07 18:33:11 UTC,NaN,NaN,NaN,NaN,1
14195,760926,30234,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:17 UTC,2015-05-07 18:33:17 UTC,NaN,NaN,NaN,NaN,1
14628,761359,0,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:23 UTC,2015-05-07 18:33:23 UTC,NaN,NaN,NaN,NaN,1
15061,761792,30235,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:29 UTC,2015-05-07 18:33:29 UTC,NaN,NaN,NaN,NaN,1
15494,762225,30236,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:36 UTC,2015-05-07 18:33:36 UTC,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921329,1669445,23346,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:17 UTC,2015-05-14 07:11:17 UTC,NaN,NaN,NaN,NaN,1
921755,1669871,4663,Compound,554,NaN,NaN,NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:30 UTC,2015-05-14 07:11:30 UTC,NaN,NaN,NaN,NaN,1
934425,1682965,11831,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,52.81,NaN,1
934426,1682966,698,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,4.94,NaN,1


In [94]:
# Replace 554 name with 'Owl, horned, flesh, raw (Alaska Native)'
still_missing.loc[still_missing['food_id']==554, 'orig_food_common_name'] = 'Owl, horned, flesh, raw (Alaska Native)'
still_missing

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
13762,760493,30233,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:11 UTC,2015-05-07 18:33:11 UTC,NaN,NaN,NaN,NaN,1
14195,760926,30234,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:17 UTC,2015-05-07 18:33:17 UTC,NaN,NaN,NaN,NaN,1
14628,761359,0,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:23 UTC,2015-05-07 18:33:23 UTC,NaN,NaN,NaN,NaN,1
15061,761792,30235,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:29 UTC,2015-05-07 18:33:29 UTC,NaN,NaN,NaN,NaN,1
15494,762225,30236,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:36 UTC,2015-05-07 18:33:36 UTC,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921329,1669445,23346,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:17 UTC,2015-05-14 07:11:17 UTC,NaN,NaN,NaN,NaN,1
921755,1669871,4663,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:30 UTC,2015-05-14 07:11:30 UTC,NaN,NaN,NaN,NaN,1
934425,1682965,11831,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,52.81,NaN,1
934426,1682966,698,Compound,16420,NaN,NaN,NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,4.94,NaN,1


In [95]:
# Replace 16420 name description with 'Soy protein concentrate, aqueous washed'
still_missing.loc[still_missing['food_id']==16420, 'orig_food_common_name'] = 'Soy protein concentrate, aqueous washed'
still_missing

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
13762,760493,30233,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:11 UTC,2015-05-07 18:33:11 UTC,NaN,NaN,NaN,NaN,1
14195,760926,30234,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:17 UTC,2015-05-07 18:33:17 UTC,NaN,NaN,NaN,NaN,1
14628,761359,0,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:23 UTC,2015-05-07 18:33:23 UTC,NaN,NaN,NaN,NaN,1
15061,761792,30235,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:29 UTC,2015-05-07 18:33:29 UTC,NaN,NaN,NaN,NaN,1
15494,762225,30236,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:36 UTC,2015-05-07 18:33:36 UTC,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921329,1669445,23346,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:17 UTC,2015-05-14 07:11:17 UTC,NaN,NaN,NaN,NaN,1
921755,1669871,4663,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:30 UTC,2015-05-14 07:11:30 UTC,NaN,NaN,NaN,NaN,1
934425,1682965,11831,Compound,16420,NaN,"Soy protein concentrate, aqueous washed",NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,52.81,NaN,1
934426,1682966,698,Compound,16420,NaN,"Soy protein concentrate, aqueous washed",NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,4.94,NaN,1


In [96]:
# Replace 16420 with 272 
still_missing['food_id'].replace(to_replace = 16420, value = 272, inplace=True)
still_missing

/var/folders/97/xy0dx75x50j0hqykjglgn13x4zqr1q/T/ipykernel_10623/3489310927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  still_missing['food_id'].replace(to_replace = 16420, value = 272, inplace=True)


,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
13762,760493,30233,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:11 UTC,2015-05-07 18:33:11 UTC,NaN,NaN,NaN,NaN,1
14195,760926,30234,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:17 UTC,2015-05-07 18:33:17 UTC,NaN,NaN,NaN,NaN,1
14628,761359,0,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:23 UTC,2015-05-07 18:33:23 UTC,NaN,NaN,NaN,NaN,1
15061,761792,30235,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:29 UTC,2015-05-07 18:33:29 UTC,NaN,NaN,NaN,NaN,1
15494,762225,30236,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-07 18:33:36 UTC,2015-05-07 18:33:36 UTC,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921329,1669445,23346,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:17 UTC,2015-05-14 07:11:17 UTC,NaN,NaN,NaN,NaN,1
921755,1669871,4663,Compound,554,NaN,"Owl, horned, flesh, raw (Alaska Native)",NaN,NaN,NaN,NaN,...,UNKNOWN,NaN,NaN,2015-05-14 07:11:30 UTC,2015-05-14 07:11:30 UTC,NaN,NaN,NaN,NaN,1
934425,1682965,11831,Compound,272,NaN,"Soy protein concentrate, aqueous washed",NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,52.81,NaN,1
934426,1682966,698,Compound,272,NaN,"Soy protein concentrate, aqueous washed",NaN,NaN,NaN,NaN,...,DATABASE,NaN,NaN,2019-08-03 15:05:06 UTC,2019-08-03 15:05:06 UTC,NaN,NaN,4.94,NaN,1


### 5) Update Content file

Recall: missing was updated to include food descriptions from Food.csv. Anything that didn't have a description went on to still_missing for identification.

We will now drop those rows with a missing description and add still_missing which has the added descriptions. 

In [97]:
# drop rows wih missing description
missing = missing.dropna(subset=['orig_food_common_name'])

In [98]:
# add still missing (updated now with names!) to missing_
missing_updated = pd.concat([missing, still_missing], ignore_index = True)

In [99]:
# Confirm there are now no missing food descriptions
missing_check = missing_updated[missing_updated['orig_food_common_name'].isnull()]
print(missing_check.shape[0], ' rows are missing common food descriptions')

0  rows are missing common food descriptions


We will repeat the above process to fill in the Content file.

In [100]:
# drop rows wih missing description
Content = Content.dropna(subset=['orig_food_common_name'])

In [101]:
# add missing_updated to Content
Content_updated = pd.concat([Content, missing_updated], ignore_index = True)

In [102]:
# Confirm there are now no missing food descriptions
Content_check = Content_updated[Content_updated['orig_food_common_name'].isnull()]
print(Content_check.shape[0], ' rows are missing common food descriptions')

0  rows are missing common food descriptions


In [103]:
# Establish a unique number for each group of food names, starting from 1
Content_updated['food_V2_ID'] = pd.factorize(Content_updated['orig_food_common_name'])[0] + 1

### 6) Export Updated Content File

Export the content file now updated and complete with food descriptions for every entry.

In [104]:
Content_updated.to_csv('FooDB/Content_updated.csv.bz2', index = None, header = True)

In [105]:
#Matches the original dimensions + 1 for the duplicate ID column
Content_updated.shape

(1871970, 27)